---
title: "Transformers: Attention Is All You Need"
author: "richsi"
date: "2024-12-30"
categories: [Attention, Paper, Transformer]
image: "images/thumbnail.png"
---

["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) by Ashish Vaswani et al., 2017.

## Introduction


In [1]:
import torch
p = torch.randn((2,2))
p

tensor([[ 0.0231,  0.2700],
        [-2.0796, -0.6109]])

## Background and Motivation

## Core Concepts
* multi-head attention
  * normal attention
* feedforward network
* encoder decoder

* Show code and visualizations

## Applications and Insights

## Conclusion